In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("ADS project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/04 22:43:56 WARN Utils: Your hostname, 专业表面7 resolves to a loopback address: 127.0.1.1; using 172.25.245.174 instead (on interface eth0)
22/09/04 22:43:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/04 22:44:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [37]:
import pandas as pd
import numpy as np

data_loc = "../data/ABS/"
tbl_income = pd.read_excel(data_loc+"annual income by SA2 regions.xlsx")
tbl_population = pd.read_excel(data_loc+"population estimates by SA2 regions.xlsx")


In [3]:
tbl_income.drop(np.arange(0,6), inplace=True)
tbl_income.drop(tbl_income.columns[[2,3,4,5,7,8,9,10,12,13,14,15,17,18,19,20,22,23,24,25,27]], axis=1, inplace=True)
tbl_income = tbl_income.reset_index(drop=True)
tbl_income.columns = ['region_code', 'region', 'total_earners', 'median_age', 'income_sum', 'income_median', 'income_mean']
tbl_income.head(10)


,region_code,region,total_earners,median_age,income_sum,income_median,income_mean
0,Australia,NaN,14425037,42,951373381486,51389,65953
1,New South Wales,NaN,4569650,42,314465961966,51818,68816
2,101021007,Braidwood,2361,51,120763285,41593,51149
3,101021008,Karabar,5100,42,338308979,61777,66335
4,101021009,Queanbeyan,6697,39,441160946,60119,65874
5,101021010,Queanbeyan - East,3393,40,237035656,63051,69860
6,101021011,Queanbeyan Region,12821,44,1050285692,71420,81919
7,101021012,Queanbeyan West - Jerrabomberra,8476,44,725602722,73851,85607
8,101031013,Bombala,1369,50,71777152,44428,52430
9,101031014,Cooma,3777,46,206500749,47029,54673


In [ ]:
tbl_income.to_csv('../data/ABS/clean_annual.csv')

In [40]:
tbl_population.drop(tbl_population.columns[0:8], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[2:22], axis=1, inplace=True)
tbl_population.drop(tbl_population.columns[3:7], axis=1, inplace=True)
# tbl_population = tbl_population.reset_index(drop=True)
tbl_population.head()

,SA2 code,SA2 name,2021,km2,persons/km2
0,101021007.0,Braidwood,4330.0,3418.4,1.3
1,101021008.0,Karabar,8546.0,7.0,1223.9
2,101021009.0,Queanbeyan,11370.0,4.8,2387.7
3,101021010.0,Queanbeyan - East,5093.0,13.0,391.7
4,101021012.0,Queanbeyan West - Jerrabomberra,12743.0,13.7,931.9


In [41]:
tbl_population.to_csv('../data/ABS/clean_population.csv')